<a href="https://colab.research.google.com/github/jtdsouza/w251-WIldTrackAI/blob/master/WildAID_PreProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization and Load Image Data

In [0]:
#Set up TF 2.0
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense, Dropout
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.initializers import lecun_normal
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras import backend as K
from keras.preprocessing import image as KImage

import cv2
import csv
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
from google.colab.patches import cv2_imshow
from collections import defaultdict
import random
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16Pre
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input as VGG19Pre
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionPre
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input as XceptionPre
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as MNPre
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet121

from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pickle

Using TensorFlow backend.


In [0]:
#Mount Google Drive - Note this mounts your personal GDrive to the directory stated
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Set up path for training/ test data. Need to map U C Berkeley - Darragh/ shared to your personal GDrive for this to work. 
path="/content/drive/My Drive/U C Berkeley - Darragh/Training Data"
test_path="/content/drive/My Drive/U C Berkeley - Darragh/Test Data"
os.listdir(path)

['Amur Tiger',
 'Bengal Tiger',
 'Cheetah',
 'Leopard',
 'Lowland Tapir',
 'Puma',
 'White Rhino',
 'Black Rhino']

In [0]:
# Function to load and individual image to a specified size.
def load_image(image,preprocessor,size=(299,299)):
  try:
    image = KImage.load_img(image,target_size=size,interpolation="nearest")
  except:
    return np.zeros(0)
  else:
    x= KImage.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    x = preprocessor(x)
    x=np.squeeze(x)
    x=x.reshape(-1)
    return x

In [0]:
# Function to load image data set from specified folder. Will be reshaped to size specified in size parameter.
# Once images are processed, writes back out to a csv file

def LoadDataSet(folder,img_output,label_output,preprocessor,size=(224,224)):
  Keys=[]
  with open(img_output, 'w') as f:
    writer = csv.writer(f)
    for species in os.listdir(folder):
      ind_path=folder+'/'+species
      print(species)

      for individual in os.listdir(ind_path):
        print_path=ind_path+'/'+individual
        #print(print_path)

        for footprint in os.listdir(print_path):
          x=load_image(os.path.join(print_path,footprint),preprocessor,(224,224))
          if x.shape[0]==0:
            continue
          else:
            key=species+"-"+individual
            Keys.append(key)
            writer.writerow(x)

  with open(label_output, 'w') as filehandle:
    filehandle.writelines("%s\n" % key for key in Keys)

In [0]:
#Point to path to write csv images and create pre-processed input for VGG16 model (224x224 image size) 
csvpath="/content/drive/My Drive/U C Berkeley - Darragh/csv"
LoadDataSet(path,os.path.join(csvpath,"Training-Images-224.csv"),os.path.join(csvpath,"Training-Labels-224.txt"),VGG16Pre,(224,224))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino


In [0]:
#Test that images were saved correctly
test_path="/content/drive/My Drive/U C Berkeley - Darragh/Test Data"
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-224.csv"),os.path.join(csvpath,"Test-Labels-224.txt"),VGG16Pre,(224,224))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino


In [0]:
#Create similar csv files for Mobilenet (224x224) and Inception net (299x299)

csvpath="/content/drive/My Drive/U C Berkeley - Darragh/csv"
test_path="/content/drive/My Drive/U C Berkeley - Darragh/Test Data"
#LoadDataSet(path,os.path.join(csvpath,"Training-Images-224.csv"),os.path.join(csvpath,"Training-Labels-224.txt"),VGG16Pre,(224,224))
#LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-224.csv"),os.path.join(csvpath,"Test-Labels-224.txt"),VGG16Pre,(224,224))
LoadDataSet(path,os.path.join(csvpath,"Training-Images-299.csv"),os.path.join(csvpath,"Training-Labels-299.txt"),preprocess_input,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-299.csv"),os.path.join(csvpath,"Test-Labels-299.txt"),preprocess_input,(299,299))
LoadDataSet(path,os.path.join(csvpath,"Train-Images-Mobile-224.csv"),os.path.join(csvpath,"Train-Labels-Mobile-224.txt"),MNPre,(224,224))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Mobile-224.csv"),os.path.join(csvpath,"Test-Labels-Mobile-224.txt"),MNPre,(224,224))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino


In [0]:
#csv files for Xception, VGG19 and Inception

csvpath="/content/drive/My Drive/U C Berkeley - Darragh/csv"
LoadDataSet(path,os.path.join(csvpath,"Training-Images-Xception-224.csv"),os.path.join(csvpath,"Training-Labels.txt"),XceptionPre,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Xception-224.csv"),os.path.join(csvpath,"Test-Labels.txt"),XceptionPre,(299,299))
LoadDataSet(path,os.path.join(csvpath,"Training-Images-VGG19-224.csv"),os.path.join(csvpath,"Training-Labels.txt"),VGG19Pre,(224,224))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-VGG19-224.csv"),os.path.join(csvpath,"Test-Labels.txt"),VGG19Pre,(224,224))
LoadDataSet(path,os.path.join(csvpath,"Training-Images-Inception-299.csv"),os.path.join(csvpath,"Training-Labels.txt"),InceptionPre,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Incception-299.csv"),os.path.join(csvpath,"Test-Labels.txt"),InceptionPre,(299,299))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
